In [118]:
import cv2
import pandas as pd

### warm cool monotone classifier

In [123]:
# pixel 단위 톤 구별 함수
def rgb_classfy(rgb):
    r = rgb[0]; g = rgb[1]; b = rgb[2]
    
    # pixel의 r, g, b의 비율을 고려하여 
    # 해당 pixel이 monotone인지 redtone인지 bluetone인지 분류
    if ((r/g) >= 0.95 and (r/g) <= 1.05) and ((g/b) >= 0.95 and (g/b) <= 1.05) and ((r/g) >= 0.95 and (r/g) <= 1.05):
        return 'mono'
    elif r < 51 and g < 51 and b < 51:
        return 'mono'
    else:
        if r > b:
            return 'red'
        else:
            return 'blue'

# image 단위 톤 구별 함수 (warm, cool, mono)
def image_tone_classifier(image_dir):

    # image의 모든 pixel을 rgb 벡터로 저장
    image = cv2.imread(image_dir)
    image = cv2.resize(image, dsize=(0, 0), fx=0.2, fy=0.2, interpolation=cv2.INTER_LINEAR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # rgb_classify를 통해 image 내의 모든 pixel을 분류
    image_list = [list(lst) for img in image for lst in img]
    rb = [rgb_classfy(rgb) for rgb in image_list]

    mono = rb.count('mono')
    red = rb.count('red')
    blue = rb.count('blue')

    # mono, red, blue에 해당하는 pixel 개수에 따라
    # 해당 image를 mono, warm, cool 중 하나로 분류
    if (red >= blue) & (red >= mono):
        return "warm"
    elif (blue > red) & (blue >= mono):
        return "cool"
    elif (mono > red) & (mono > blue):
        return "mono"

In [114]:
import os
#인플루언서 리스트
os.chdir('C:/KUIAI2023/influencer_sample')
influencer = os.listdir() #인스타 계정 리스트 저장
#경로 내 파일명 추출
images = {} #key: 계정이름, value: 해당 계정의 이미지 파일명 리스트
for i in influencer:
    os.chdir(f'C:/KUIAI2023/influencer_sample/{i}/images')
    files = os.listdir()
    images[i] = files

In [115]:
tone = {} #key: 계정이름, value: mono, warm, cool 의 image 비율 리스트
total_tone = []

# 각 인플루언서에 대해 mono, warm, cool인 이미지의 비율을 리스트로 저장
# ex) tone = {"@skuukzky":[0.1, 0.6, 0.4], ...}
for i in influencer:
    mono_cnt = 0
    warm_cnt = 0
    cool_cnt = 0
    img_cnt = len(images[i])
    dir_i = f'C:/KUIAI2023/influencer_sample/{i}/images/'
    for img in images[i]:
        img_tone = image_tone_classifier(dir_i+img)
        if img_tone == "mono":
            mono_cnt += 1
        elif img_tone == "warm":
            warm_cnt += 1
        elif img_tone == "cool":
            cool_cnt += 1
    tone[i] = [mono_cnt/img_cnt, warm_cnt/img_cnt, cool_cnt/img_cnt]
    total_tone.append(tone[i])

C:\Users\황민아\AppData\Local\Temp\ipykernel_26388\3174003528.py:5: RuntimeWarning: divide by zero encountered in ubyte_scalars
  if ((r/g) >= 0.95 and (r/g) <= 1.05) and ((g/b) >= 0.95 and (g/b) <= 1.05) and ((r/g) >= 0.95 and (r/g) <= 1.05):
C:\Users\황민아\AppData\Local\Temp\ipykernel_26388\3174003528.py:5: RuntimeWarning: invalid value encountered in ubyte_scalars
  if ((r/g) >= 0.95 and (r/g) <= 1.05) and ((g/b) >= 0.95 and (g/b) <= 1.05) and ((r/g) >= 0.95 and (r/g) <= 1.05):


In [116]:
# 각 인플루언서에 대한 tone ratio list
df = pd.DataFrame()
df['id'] = influencer
df['tone'] = total_tone
df.to_csv("C:/KUIAI2023/tone.csv")